# Fitting Vine Copulas to Incomplete Data

This notebook demonstrates how to fit vine copulas when different variable pairs have different numbers of observations (missing data).

## The Problem

Standard vine copula fitting requires complete cases — observations where all variables are present. But in practice:
- Some variable pairs may have more observations than others
- Requiring complete cases discards valuable pairwise information
- Higher trees need fewer variables, so may have more usable data

## The Solution

The `fit_vine_incomplete` function:
1. Uses all available pairwise observations for Tree 1
2. Uses all available triple observations for Tree 2
3. Uses all available k-tuple observations for Tree k-1
4. Automatically truncates (sets to independence) when observations drop below a threshold

## Realistic Workflow

This example simulates data in **original space** (not uniform copula space) to mirror real-world usage:
1. Simulate from known marginal distributions + vine copula dependence
2. Introduce missing values (different variables have different observation counts)
3. Fit marginals per-column using available data
4. Transform to pseudo-observations with `to_pseudo_obs_incomplete()`
5. Fit the vine copula on incomplete pseudo-observations

In [1]:
import numpy as np
from scipy import stats
import pyvinecopulib as pv
from pyvinecopulib import (
  fit_vine_incomplete,
  get_complete_counts,
  to_pseudo_obs_incomplete,
)

## Define True Model: Marginals + Vine Copula

We define:
1. **Marginal distributions** for each variable (Normal, Student-t, Gamma, Beta)
2. **Vine copula** capturing the dependence structure

In [2]:
np.random.seed(42)

# True marginal distributions (diverse shapes)
true_marginals = [
  stats.norm(loc=100, scale=15),  # Variable 0: Normal (e.g., price level)
  stats.t(df=5, loc=0.05, scale=0.1),  # Variable 1: Student-t (e.g., returns)
  stats.gamma(a=2, scale=0.02),  # Variable 2: Gamma (e.g., spread)
  stats.beta(a=2, b=5),  # Variable 3: Beta (e.g., allocation)
]
var_names = ["price", "return", "spread", "weight"]

# Define pair-copulas with known dependence
bicop_gauss = pv.Bicop(family=pv.gaussian, parameters=np.array([[0.7]]))
bicop_clayton = pv.Bicop(family=pv.clayton, parameters=np.array([[2.0]]))

# Build a 4-dimensional vine
pcs = [
  [bicop_gauss, bicop_gauss, bicop_gauss],  # Tree 1: 3 edges
  [bicop_clayton, bicop_clayton],  # Tree 2: 2 edges
  [bicop_gauss],  # Tree 3: 1 edge
]

# C-vine structure with variable 0 as root
structure = np.array(
  [[1, 1, 1, 1], [2, 2, 2, 0], [3, 3, 0, 0], [4, 0, 0, 0]], dtype=np.uint64
)

true_vine = pv.Vinecop.from_structure(matrix=structure, pair_copulas=pcs)
print("True vine copula:")
print(true_vine)
print("\nTrue marginals:")
for i, (name, marginal) in enumerate(zip(var_names, true_marginals)):
  print(
    f"  {name}: {marginal.dist.name}(mean={marginal.mean():.3f}, std={marginal.std():.3f})"
  )

True vine copula:
<pyvinecopulib.Vinecop> Vinecop model with 4 variables
tree edge conditioned variables conditioning variables var_types   family rotation parameters  df  tau 
   1    1                  4, 1                             c, c Gaussian        0       0.70 1.0 0.49 
   1    2                  3, 1                             c, c Gaussian        0       0.70 1.0 0.49 
   1    3                  2, 1                             c, c Gaussian        0       0.70 1.0 0.49 
   2    1                  4, 2                      1      c, c  Clayton        0       2.00 1.0 0.50 
   2    2                  3, 2                      1      c, c  Clayton        0       2.00 1.0 0.50 
   3    1                  4, 3                   2, 1      c, c Gaussian        0       0.70 1.0 0.49 


True marginals:
  price: norm(mean=100.000, std=15.000)
  return: t(mean=0.050, std=0.129)
  spread: gamma(mean=0.040, std=0.028)
  weight: beta(mean=0.286, std=0.160)


## Simulate Data in Original Space

We simulate from the vine copula (uniform space), then apply inverse CDFs to get data in original space. This mimics real data with arbitrary marginal distributions.

In [3]:
# Simulate from vine copula (uniform space)
n_samples = 1000
U_complete = true_vine.simulate(n_samples)

# Transform to original space via inverse CDFs
X_complete = np.column_stack(
  [marginal.ppf(U_complete[:, i]) for i, marginal in enumerate(true_marginals)]
)

print(f"Data in original space: {X_complete.shape}")
print("\nSample statistics (complete data):")
for i, name in enumerate(var_names):
  print(
    f"  {name}: mean={X_complete[:, i].mean():.3f}, std={X_complete[:, i].std():.3f}"
  )

# Introduce missing values (different start dates for different variables)
# This mimics real scenarios like: HY data from 2001, CLO from 2011, RMBS from 2014
X = X_complete.copy()
X[:500, 2] = np.nan  # "spread" missing for first 50% (started later)
X[:300, 3] = np.nan  # "weight" missing for first 30% (started even later)

print("\nMissing pattern (simulating different start dates):")
for i, name in enumerate(var_names):
  n_available = (~np.isnan(X[:, i])).sum()
  n_missing = np.isnan(X[:, i]).sum()
  print(f"  {name}: {n_available} available, {n_missing} missing")

Data in original space: (1000, 4)

Sample statistics (complete data):
  price: mean=100.340, std=14.782
  return: mean=0.051, std=0.122
  spread: mean=0.039, std=0.027
  weight: mean=0.285, std=0.155

Missing pattern (simulating different start dates):
  price: 1000 available, 0 missing
  return: 1000 available, 0 missing
  spread: 500 available, 500 missing
  weight: 700 available, 300 missing


## Fit Marginals Per-Column

Since each variable has different observation counts, we fit marginals using only the available data for that column. Here we use scipy's distribution fitting.

In [4]:
# Fit marginals on available data per column
fitted_marginals = []
print("Fitting marginals per column:")
print("=" * 60)

for i, name in enumerate(var_names):
  valid_mask = ~np.isnan(X[:, i])
  x_valid = X[valid_mask, i]

  # Fit normal distribution (simple example - could use KDE or other)
  mu, sigma = stats.norm.fit(x_valid)
  fitted = stats.norm(loc=mu, scale=sigma)
  fitted_marginals.append(fitted)

  print(
    f"  {name}: fitted Normal(μ={mu:.3f}, σ={sigma:.3f}) on {len(x_valid)} obs"
  )

Fitting marginals per column:
  price: fitted Normal(μ=100.340, σ=14.782) on 1000 obs
  return: fitted Normal(μ=0.051, σ=0.122) on 1000 obs
  spread: fitted Normal(μ=0.042, σ=0.029) on 500 obs
  weight: fitted Normal(μ=0.293, σ=0.156) on 700 obs


## Transform to Pseudo-Observations

Use `to_pseudo_obs_incomplete()` to convert data to uniform [0,1] scale. This handles missing values automatically — each column is transformed using only its available observations.

In [6]:
# Transform to pseudo-observations (handles NaN per-column)
U = to_pseudo_obs_incomplete(X)

print("Pseudo-observations (first 5 rows):")
print(U[:5])
print(f"\nShape: {U.shape}")
print(f"NaN preserved: {np.isnan(U).sum()} missing values in pseudo-obs")

Pseudo-observations (first 5 rows):
[[0.41258741 0.53146853        nan        nan]
 [0.17382617 0.33566434        nan        nan]
 [0.64935065 0.53546454        nan        nan]
 [0.44655345 0.13386613        nan        nan]
 [0.01198801 0.05094905        nan        nan]]

Shape: (1000, 4)
NaN preserved: 800 missing values in pseudo-obs


## Examine Data Availability Per Edge

The `get_complete_counts` function shows how many observations are available for each edge in the vine structure.

In [7]:
counts = get_complete_counts(U, structure)

print("Complete observations per edge:")
print("=" * 50)
for (tree, edge, cond), count in sorted(counts.items()):
  cond_str = f" | {cond}" if cond else ""
  print(f"  Tree {tree}, Edge {edge}{cond_str}: {count} observations")

Complete observations per edge:
  Tree 0, Edge 0: 700 observations
  Tree 0, Edge 1: 500 observations
  Tree 0, Edge 2: 1000 observations
  Tree 1, Edge 0 | (0,): 700 observations
  Tree 1, Edge 1 | (0,): 500 observations
  Tree 2, Edge 0 | (1, 0): 500 observations


## Compare: Complete Cases vs Incomplete Data Fitting

Complete-case fitting discards rows with any missing value (50% of data). Incomplete-data fitting uses all available observations per edge.

In [13]:
# Standard approach: complete cases only
complete_mask = ~np.any(np.isnan(U), axis=1)
n_complete = complete_mask.sum()
print(
  f"Complete cases: {n_complete} / {len(U)} ({100 * n_complete / len(U):.1f}%)"
)

vine_complete = pv.Vinecop.from_data(U[complete_mask], matrix=structure)
print("\nVine fitted on complete cases only:")
print(vine_complete)

Complete cases: 500 / 1000 (50.0%)

Vine fitted on complete cases only:
<pyvinecopulib.Vinecop> Vinecop model with 4 variables
tree edge conditioned variables conditioning variables var_types   family rotation parameters  df  tau 
   1    1                  4, 1                             c, c Gaussian        0       0.70 1.0 0.50 
   1    2                  3, 1                             c, c Gaussian        0       0.72 1.0 0.51 
   1    3                  2, 1                             c, c Gaussian        0       0.72 1.0 0.51 
   2    1                  4, 2                      1      c, c  Clayton        0       1.81 1.0 0.48 
   2    2                  3, 2                      1      c, c  Clayton        0       1.81 1.0 0.48 
   3    1                  4, 3                   2, 1      c, c Gaussian        0       0.67 1.0 0.46 



In [14]:
# Our approach: use all available data per edge
vine_incomplete = fit_vine_incomplete(U, min_obs=100, structure=structure)
print("Vine fitted on incomplete data (all available observations per edge):")
print(vine_incomplete)

Vine fitted on incomplete data (all available observations per edge):
<pyvinecopulib.Vinecop> Vinecop model with 4 variables
tree edge conditioned variables conditioning variables var_types   family rotation parameters  df  tau 
   1    1                  4, 1                             c, c Gaussian        0       0.71 1.0 0.50 
   1    2                  3, 1                             c, c Gaussian        0       0.72 1.0 0.51 
   1    3                  2, 1                             c, c Gaussian        0       0.72 1.0 0.51 
   2    1                  4, 2                      1      c, c  Clayton        0       1.90 1.0 0.49 
   2    2                  3, 2                      1      c, c  Clayton        0       1.81 1.0 0.48 
   3    1                  4, 3                   2, 1      c, c Gaussian        0       0.66 1.0 0.46 



## Key Observations

1. **Complete cases**: Uses only 500 rows (50% of data)
2. **Incomplete-data fit**: Uses 700/500/1000 observations per edge
3. **Efficiency**: Edges with more data (e.g., price-return with 1000 obs) get better estimates

## Automatic Truncation with Severe Missingness

When data become too sparse, edges are automatically set to independence.

In [15]:
# Create data with severe missingness (minimal overlap)
np.random.seed(123)
U_severe = true_vine.simulate(1000)
X_severe = np.column_stack(
  [marginal.ppf(U_severe[:, i]) for i, marginal in enumerate(true_marginals)]
)

# Severe missingness: variables 1 and 2 barely overlap
X_severe[:500, 1] = np.nan  # return: available rows 500-999
X_severe[550:, 2] = np.nan  # spread: available rows 0-549
# Overlap of return & spread: only rows 500-549 (50 obs!)

print("Severe missing pattern:")
for i, name in enumerate(var_names):
  n_available = (~np.isnan(X_severe[:, i])).sum()
  print(f"  {name}: {n_available} observations available")

mask_1 = ~np.isnan(X_severe[:, 1])
mask_2 = ~np.isnan(X_severe[:, 2])
print(f"\nreturn & spread overlap: {(mask_1 & mask_2).sum()} observations")

Severe missing pattern:
  price: 1000 observations available
  return: 500 observations available
  spread: 550 observations available
  weight: 1000 observations available

return & spread overlap: 50 observations


In [16]:
# Transform and fit with automatic truncation
U_severe = to_pseudo_obs_incomplete(X_severe)
vine_truncated = fit_vine_incomplete(U_severe, min_obs=100, structure=structure)

print("Vine with automatic truncation:")
print(vine_truncated)

# Show which edges were truncated
counts_severe = get_complete_counts(U_severe, structure)
print("\nData availability (severe missingness):")
print("=" * 50)
for (tree, edge, cond), count in sorted(counts_severe.items()):
  status = "✓" if count >= 100 else "✗ (truncated)"
  cond_str = f" | {cond}" if cond else ""
  print(f"  Tree {tree}, Edge {edge}{cond_str}: {count:4d} obs {status}")

Vine with automatic truncation:
<pyvinecopulib.Vinecop> Vinecop model with 4 variables
tree edge conditioned variables conditioning variables var_types       family rotation parameters  df  tau 
   1    1                  4, 1                             c, c     Gaussian        0       0.71 1.0 0.51 
   1    2                  3, 1                             c, c     Gaussian        0       0.73 1.0 0.52 
   1    3                  2, 1                             c, c     Gaussian        0       0.72 1.0 0.51 
   2    1                  4, 2                      1      c, c          Joe      180       2.94 1.0 0.51 
   2    2                  3, 2                      1      c, c Independence                         0.00 
   3    1                  4, 3                   2, 1      c, c Independence                         0.00 


Data availability (severe missingness):
  Tree 0, Edge 0: 1000 obs ✓
  Tree 0, Edge 1:  550 obs ✓
  Tree 0, Edge 2:  500 obs ✓
  Tree 1, Edge 0 | (0,):  50

## Summary

This notebook demonstrated a realistic workflow for fitting vine copulas to incomplete data:

### Workflow
1. **Start with data in original space** (prices, returns, etc.)
2. **Fit marginals per-column** using available observations
3. **Transform to pseudo-observations** with `to_pseudo_obs_incomplete()`
4. **Fit vine copula** with `fit_vine_incomplete()`

### Key Functions

```python
# Transform data with missing values to pseudo-observations
to_pseudo_obs_incomplete(data)  # handles NaN per-column

# Fit vine copula on incomplete data
fit_vine_incomplete(
    data,                    # (n, d) array with np.nan for missing
    min_obs=100,             # Minimum observations to fit an edge
    structure=None,          # R-vine matrix (auto-selected if None)
    family_set=None,         # Copula families to consider
    trunc_lvl=None           # Maximum truncation level
)

# Diagnostic: observation counts per edge
get_complete_counts(data, structure)
```

### Benefits
1. **Maximum data utilization**: Each edge uses all available observations
2. **Automatic truncation**: Sparse edges set to independence
3. **Diagnostics**: `get_complete_counts()` exposes per-edge data availability